# Entrega 1: Visualización del conjunto de datos

 Contexto: deseamos predecir cuánto tiempo estarán en tierra aquellos aviones en puntos de espera, es
 decir, cuánto tiempo transcurre entre el punto de espera y el despegue. Por ahora, en lugar de comprobar
 puntos de espera consideraremos tiempo desde la última vez que está parado (obviamente, en tierra) hasta
 que está en el aire.

## Lectura y procesamiento de los datos

El conjunto de datos de esta primera entrega corresponde a los registros de los vuelos de una semana. Estos se encuentran en un archivo csv y están codificados.

Lo primero que haremos será leer el csv (se encuentra comprimido en un archivo tar), procesar y guardar los almacenar los resultados en la carpeta datos:

In [2]:
from preprocess.reader import read_data

TAR_PATH = "C:/Users/bryan/PycharmProjects/despegues/src/202412010000_202412072359.tar"
FILE_NAME = "202412010000_202412072359.csv"
read_data(TAR_PATH, FILE_NAME) # Se aplica paralelismo, sim embargo, sigue tardando un par de horas

        ts_kafka               message
0  1733011203260  jUoZIupCiFgBPAjykJ4=
1  1733011203260  oAAUllilYmBIYPDDtec=
2  1733011203260  oAAZEMMJ5TEgP//kyUg=
3  1733011203260  jQIBK/gjAAIASbh2gt8=
4  1733011203260          XTRCE1QV9w==


Ahora los datos ya se encuentran procesados y en formato parquet (para observar como funciona por dentro recomendamos ver los distintos módulos del proyecto).

A continuación leemos los archivos parquet y los almacenamos en DataFrames de Pandas:

In [ ]:
import pandas as pd
import glob
import plotly.express as px
from preprocess.utilities import stringToNan

# Ruta de los archivos parquet
file_pattern = "./preprocess/data/part_*.parquet"
output_file = "./preprocess/data/primeraSemanaDash.parquet"

# Obtener lista de archivos parquet
file_list = sorted(glob.glob(file_pattern))

# Seleccionar solo las columnas necesarias
selected_columns = [
    "Timestamp (date)", "ICAO", "Callsign", "Flight status", "Speed",
    "Altitude (ft)", "lat", "lon", "Typecode", "TurbulenceCategory", "Downlink Format"
]

# Cargar, filtrar y limpiar datos en un bucle
df_list = [stringToNan(pd.read_parquet(file)[selected_columns]) for file in file_list]

# Concatenar todos los DataFrames
df = pd.concat(df_list, ignore_index=True)

# Convertir Timestamp a datetime
df['Timestamp (date)'] = pd.to_datetime(df['Timestamp (date)'])

# Extraer hora y día de la semana
df['hour'] = df['Timestamp (date)'].dt.floor('h')
df['day_of_week'] = df['Timestamp (date)'].dt.strftime('%a')

In [3]:
# Guardar el DataFrame combinado en un archivo Parquet
save_file = "./data/primeraSemanaDash.parquet"
df.to_parquet(save_file)
print("Archivo parquet generado y guardado.")

Archivo parquet generado y guardado.


LEER SOLO ESTE

In [1]:
import pandas as pd

df = pd.read_parquet("./data/primeraSemanaDash.parquet", engine="pyarrow")

## Ejercicio 1 Con los datos proporcionados, genera visualizaciones que permitan entender el conjunto de datos. 

En particular,

 (a) tráfico aéreo por horas, distinguiendo aviones en tierra y en el aire;

 (b) distribución de los tiempos de espera

 Histograma para ver cómo se distribuyen los tiempos de espera en general.
 Boxplot para identificar la mediana, los cuartiles y posibles valores atípicos.
 Mapa de calor para ver qué horas del día tienen mayor tiempo de espera.

 (c) (opcional) completa con otras visualizaciones que consideres interesantes para el problema pro
puesto;

 (d) (opcional) filtra, según la gráfica, por días, por franjas horarias, por vuelos, por tipo de avión,
 por pistas, etc.

In [ ]:
import pandas as pd
from preprocess.dataframe_processor import DataframeProcessor
import preprocess.utilities as ut

In [ ]:
 # Conseguimos el df con todos los datos necesarios->flight status en todos los callsign
df1 = DataframeProcessor.getVelocities(df)
df2 = DataframeProcessor.getFlights(df)

df1_s = df1.sort_values(["Timestamp (date)", "ICAO"])
df2_s = df2.sort_values(["Timestamp (date)", "ICAO"])

t = pd.Timedelta('10 minute')
dff = pd.merge_asof(df1_s, df2_s, on="Timestamp (date)", by="ICAO", direction="nearest", tolerance=t)

# Ensure timestamp is in datetime format
dff['Timestamp (date)'] = pd.to_datetime(dff['Timestamp (date)'])

# Extract hour
dff = ut.extractHour(dff)

# Day of the week
dff = ut.extractDaysOfTheWeek(dff)

df_status = df.groupby(['hour', 'Flight status', 'Callsign']).size().unstack(fill_value=0)
# Sumammos el número de vuelos, no el número de mensajes
df_status['count_nonzero'] = (df_status.ne(0)).sum(axis=1)
df_status = df_status.reset_index()

# Summarize data: count_nonzero per hour divided by Flight status
df_status = df_status.groupby(['hour', 'Flight status'])['count_nonzero'].sum().reset_index()

In [ ]:
# no ejecutar esta
traffic_by_hour = df.groupby(['hour', 'Flight status']).size().unstack(fill_value=0)
traffic_by_hour_reset = traffic_by_hour.reset_index()
traffic_melted = traffic_by_hour_reset.melt(id_vars=['hour'], var_name='Flight Status', value_name='Count')

In [ ]:
# Create interactive stacked bar chart using Plotly
fig_bar = px.bar(
    df_status,
    y='hour',
    x='Count',
    color='Flight Status',
    title="Hourly Air Traffic (On-Ground vs Airborne)",
    labels={'hour': 'Hour', 'Count': 'Number of Flights'},
    barmode='stack'
)

# Update layout for better visualization
fig_bar.update_layout(
    xaxis_title="Hour",
    yaxis_title="Number of Flights",
    xaxis_tickangle=-45,
    legend_title="Flight Status"
)

# Show interactive chart
fig_bar.show()

1.b distribución de los tiempos de espera

In [ ]:
from preprocess.dataframe_processor import DataframeProcessor

df1 = DataframeProcessor.getVelocities(df)
df2 = DataframeProcessor.getFlights(df)

df1_s = df1.sort_values(["Timestamp (date)", "ICAO"])
df2_s = df2.sort_values(["Timestamp (date)", "ICAO"])

t = pd.Timedelta('10 minute')
dff = pd.merge_asof(df1_s, df2_s, on="Timestamp (date)", by="ICAO", direction="nearest", tolerance=t)

In [ ]:
# Ensure data is sorted by Flight ID and timestamp
dff = dff.sort_values(by=["Callsign", "Timestamp (date)"])

# Separate on-ground and airborne events
on_ground = dff[(dff["Flight status"] == "on-ground") & (dff["Speed"]==0)].groupby("Callsign")["Timestamp (date)"].min()
airborne = dff[dff["Flight status"] == "airborne"].groupby("Callsign")["Timestamp (date)"].min()

In [ ]:
on_ground = pd.DataFrame(on_ground)
on_ground.columns = ["ts ground"]

In [ ]:
airborne = pd.DataFrame(airborne)
airborne.columns = ["ts airborne"]

In [ ]:
df_wait_times = on_ground.merge(airborne, how="inner", on="Callsign")
df_wait_times = df_wait_times[df_wait_times["ts airborne"] > df_wait_times["ts ground"]]
df_wait_times["Wait time"] = df_wait_times["ts airborne"] - df_wait_times["ts ground"]
df_wait_times["Wait time (s)"] = df_wait_times["Wait time"].dt.total_seconds()
df_wait_times['day_of_week'] = df_wait_times['ts ground'].dt.strftime('%a')

In [ ]:
# Create a histogram
fig_hist = px.histogram(df_wait_times, x="Wait time (s)", nbins=10, title="Wait Time Distribution")

# Show the plot
fig_hist.show()

In [ ]:
# Crear un boxplot
fig_box = px.box(df_wait_times, y="Wait time (s)", title="Boxplot de Valores")  # points="all" muestra los valores atípicos

# Mostrar la gráfica
fig_box.show()

MAPAS DE CALOR POR HORAS DEL DIA

In [ ]:
# Extraer fecha y hora
df_wait_times["Date"] = df_wait_times["ts ground"].dt.date
df_wait_times["Hour"] = df_wait_times["ts ground"].dt.hour

# Crear tabla pivote para el heatmap
heatmap_data = df_wait_times.pivot_table(index="Date", columns="Hour", values="Wait time (s)", aggfunc="mean")

# Crear el mapa de calor con Plotly
fig_heatmap = px.imshow(
    heatmap_data.values,
    labels=dict(x="Hour", y="Date", color="Tiempo de espera (s)"),
    x=heatmap_data.columns,
    y=heatmap_data.index,
    color_continuous_scale="RdYlGn_r",
    title="Mapa de Calor del Tiempo de Espera por Hora y Día del Mes"
)

# Mostrar el gráfico
fig_heatmap.show()


In [ ]:
# Gráfica por horas de aviones en tierra y aterrizados
def graph_hourly_flight_status(df):
    """
    Generates an interactive stacked bar chart showing the hourly distribution of flights
    based on their status (e.g., on-ground or airborne).

    Args:
        df (pd.DataFrame): A DataFrame containing flight data with at least two columns:
            - 'Timestamp (date)': Datetime column representing flight event timestamps.
            - 'Flight status': Categorical column indicating the flight's status.

    Returns:
        plotly.graph_objects.Figure: A Plotly figure object displaying the stacked bar chart.
    """
    # Ensure timestamp is in datetime format
    df['Timestamp (date)'] = pd.to_datetime(df['Timestamp (date)'])

    # Extract hour
    df['hour'] = df['Timestamp (date)'].dt.floor('H')

    # Group by hour and flight status
    traffic_by_hour = df.groupby(['hour', 'Flight status']).size().unstack(fill_value=0)

    # Reset index for plotting
    traffic_by_hour_reset = traffic_by_hour.reset_index()

    # Melt the DataFrame for Plotly
    traffic_melted = traffic_by_hour_reset.melt(id_vars=['hour'], var_name='Flight Status', value_name='Count')

    # Create interactive stacked bar chart using Plotly
    fig = px.bar(
        traffic_melted,
        y='hour',
        x='Count',
        color='Flight Status',
        title="Hourly Air Traffic (On-Ground vs Airborne)",
        labels={'hour': 'Hour', 'Count': 'Number of Flights'},
        barmode='stack'
    )

    # Update layout for better visualization
    fig.update_layout(
        xaxis_title="Hour",
        yaxis_title="Number of Flights",
        xaxis_tickangle=-45,
        legend_title="Flight Status"
    )

    return fig

# Sacar df de los tiempos de espera
def df_wait_times(df):
    # Extraemos las columnas necesarias y creamos un df nuevo
    df1 = DataframeProcessor.getVelocities(df)
    df2 = DataframeProcessor.getFlights(df)

    df1_s = df1.sort_values(["Timestamp (date)", "ICAO"])
    df2_s = df2.sort_values(["Timestamp (date)", "ICAO"])

    t = pd.Timedelta('10 minute')
    dff = pd.merge_asof(df1_s, df2_s, on="Timestamp (date)", by="ICAO", direction="nearest", tolerance=t)

    # Ensure data is sorted by Flight ID and timestamp
    dff = dff.sort_values(by=["Callsign", "Timestamp (date)"])

    # Separate on-ground and airborne events
    on_ground = dff[(dff["Flight status"] == "on-ground") & (dff["Speed"]==0)].groupby("Callsign")["Timestamp (date)"].min()
    airborne = dff[dff["Flight status"] == "airborne"].groupby("Callsign")["Timestamp (date)"].min()

    # Create new dataframes and rename timestamp columns
    on_ground = pd.DataFrame(on_ground)
    on_ground.columns = ["ts ground"]

    airborne = pd.DataFrame(airborne)
    airborne.columns = ["ts airborne"]

    # Merge them into a new dataframe and extract the waiting seconds
    df_wait_times = on_ground.merge(airborne, how="inner", on="Callsign")
    df_wait_times = df_wait_times[df_wait_times["ts airborne"] > df_wait_times["ts ground"]]
    df_wait_times["Wait time"] = df_wait_times["ts airborne"] - df_wait_times["ts ground"]
    df_wait_times["Wait time (s)"] = df_wait_times["Wait time"].dt.total_seconds()

    return df_wait_times

def histogram_wait_times(df):
    fig_hist = px.histogram(df, x="Wait time (s)", nbins=10, title="Wait Time Distribution")
    return fig_hist

def boxplot_wait_times(df):
    fig_box = px.box(df_wait_times, y="Wait time (s)", title="Boxplot de Valores")
    return fig_box

def heatmap_wait_times(df):
    # Extraer fecha y hora
    df["Date"] = df["ts ground"].dt.date
    df["Hour"] = df["ts ground"].dt.hour

    # Crear tabla pivote para el heatmap
    heatmap_data = df.pivot_table(index="Date", columns="Hour", values="Wait time (s)", aggfunc="mean")

    # Crear el mapa de calor con Plotly
    fig_heatmap = px.imshow(
        heatmap_data.values,
        labels=dict(x="Hour", y="Date", color="Tiempo de espera (s)"),
        x=heatmap_data.columns,
        y=heatmap_data.index,
        color_continuous_scale="RdYlGn_r",
        title="Mapa de Calor del Tiempo de Espera por Hora y Día del Mes"
    )

    return fig_heatmap


In [ ]:
import dash
from dash import dcc, html, Input, Output
import plotly.express as px
import pandas as pd
import numpy as np

In [ ]:
# Crear la aplicación Dash
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Air Traffic Dashboard"),

    dcc.Dropdown(
        id='day-dropdown',
        options=[{'label': day, 'value': day} for day in df['day_of_week'].unique()],
        value=df['day_of_week'].unique()[0],  # Valor por defecto
        clearable=False
    ),

    dcc.Graph(id='bar-graph'),
    dcc.Graph(id='hist-graph'),
    dcc.Graph(id='box-graph'),
    dcc.Graph(id='heatmap-graph', figure=heatmap_wait_times(df_wait_times(df)))
])

# Callback para actualizar gráficos basados en el día seleccionado
@app.callback(
    [Output('bar-graph', 'figure'),
     Output('hist-graph', 'figure'),
     Output('box-graph', 'figure')],

    [Input('day-dropdown', 'value')]
)
def update_graphs(selected_day):
    df1 = df[df['day_of_week'] == selected_day]
    df2 = df_wait_times(df1)

    fig_bar = graph_hourly_flight_status(df1)
    fig_hist = histogram_wait_times(df2)
    fig_box = boxplot_wait_times(df2)

    return fig_bar, fig_hist, fig_box

if __name__ == '__main__':
    app.run_server(debug=True)

 ## Ejercicio 2 Con los datos proporcionados, visualiza la información en un mapa.
 
 En concreto,

 (a) sitúa las pistas;

 (b) sitúa los aviones;

 (c) indica si están en tierra o en el aire;

 (d) (opcional) indica dirección;

 (e) (opcional) indica velocidad;

 (f) (opcional) indica trayectorias por avión y vuelo


In [ ]:
from preprocess.dataframe_processor import DataframeProcessor
from visualization.maps import Maps

Para visualizar los datos en mapas, primero nos quedamos solo con la información de los vuelos. Después eliminamos de los datos aquellos aviones que consideramos "ouliers" siguiendo el siguiente criterio:

- `Un vuelo es outlier si recorre una distancia mayor a 200km en menos de 10 minutos.`

In [ ]:
df_maps = DataframeProcessor.getFlightsInfo(df)
df_maps = DataframeProcessor.removeOutlierFlights(df_maps)

### 1. Mapa de dispersión de los estados de vuelo

En este mapa se posicionan todos los mensajes detectados en un periodo de tiempo determinado, permitiendo distinguir si la aeronave está __en tierra (on-ground)__ o __en el aire (airborne)__. Haciendo hover sobre los puntos se puede observar información adicional (ICAO, Callsign, etc.). Asimismo, se posicionan el __radar__ y las cuatro __pistas__ del aeropuerto. Por lo tanto, se trata de una primera aproximación para observar el tráfico aéreo.

In [ ]:
m = Maps.positionsScatterMap(df_maps)
m

In [ ]:
# PARA GUARDAR ESTOS MAPAS COMO HTML
m.save("nombre.html")

### 2. Mapa de calor

Esta visualización deriva de la anterior, se trata de un mapa de calor animado que permite monitorizar la densidad de mensajes en ventanas de tiempo muy reducidas (hemos fijado los frames a cinco minutos). De este modo, podemos observar el tráfico aéreo de forma mucho más precisa y en contexto. Se incluyen tanto las señales emitidas por aeronaves situadas tanto en tierra como en el aire.

In [ ]:
m = Maps.positionsHeatMap(df_maps)
m

### 3. Trayectorias por tipo de avión

El siguiente mapa permite visualizar las __trayectorias__ de las aeronaves por ICAO e identificador de vuelo distinguiendo por categoría de turbulencia. Asimismo, permite la opción de filtrar las trayectorias por despegues, aterrizajes y otros vuelos ajenos al aeropuerto. Es preciso indicar que las trayectorias se han generalizado a rectas con la finalidad de poder abarcar ventanas temporales más grandes. El punto en el final de uno de los extremos indica la __dirección__ del vuelo.

In [ ]:
m = Maps.trajectoriesMap(df_maps)
m

### 4. Trayectorias con velocidad

Este mapa permite visualizar las **trayectorias** de forma más detallada y precisa, pero está orientado a franjas temporales más pequeñas. La **velocidad** viene indicada por el color de la recta en ese punto de la trayectoria, mientras que la **dirección** se indica con un punto en uno de los extremos. Cabe destacar que la velocidad de las señales tiene como unidad el nudo (1 kt=1.852 km/h), así que hemos hecho la conversión a km/h.

In [ ]:
m = Maps.detailedTrajectoriesMap(df_maps)
m

### 5. Trayectorias con altitud

En este mapa, al igual que en el de velocidades, se representan las trayectorias detalladas de cada vuelo. El color indica la altitud de cada aeronave a lo largo de su trayectoria, mientras que la dirección se indica con un punto en uno de los extremos. La altitud se muestra en pies (ft).

In [ ]:
m = Maps.altitudesMap(df_maps)
m